# GeoSpatial 2: satellite image processing
---

Required packages:
* numpy
* pillow/PIL
* matplotlib

---

**Which example of Deforestation are we examining?**    
Change from 1984 to 2022 in a 60 square km region of interest (ROI) originally covered mainly by tropical dry forest at the Brazil-Bolivia border within the Amazon River basin. 
    
More context available from NASA's "Mapping the Amazon" series, which happens to feature the wider ROI -> https://earthobservatory.nasa.gov/images/145649/mapping-the-amazon

**What scientific approaches are we taking?**    
Satellite image processing and classification (hand-crafted/experimental)

**What outputs will we develop?**       
Spectral vegetation indices (NDVI) and thematic maps (Land Cover).

**What will our outputs tell us?**    
Help quantitatively detect and measure Land Cover and Land Use (LCLU) change in this ROI, particularly the clearance of original tropical dry forest since 1984.

**Beyond the well-known Eco impacts of Deforestation, what makes this example significant?**      
Exemplifies the contrasting historical LCLU change in a biome as a consequence of country boundaries and local differences, e.g. in Forest Governance, Stewardship etc.      
Possible view of the ROI at ground-level -> https://andrebaertschi.photoshelter.com/image/I0000Nax5X69EnEA

Can highlight issues of REDD project efficacy (Reducing Emissions From Deforestation and Forest Degradation) and carbon offsetting generally. The Noel Kempff Mercado National Park, which takes up a sizeable portion in the bottom-right of the ROI, is part of one of the world's first large-scale REDD projects, receiving investment from 3 energy companies in exchange for rights to apparently 51% of carbon credits generated, and had been accused of significant flaws -> https://www.theguardian.com/environment/2010/mar/11/greenwash-noel-kempff-forests


**Data Source - Satellite Imagery**
* 4x files:
    * `ROI_1984_nirband4.tif` - clip of Landsat 5 TM Scene 230/69, Near Infrared Band (B4) (Aug 13th, 1984)
    * `ROI_1984_redband3.tif` - clip of Landsat 5 TM Scene 230/69, Red Band (B3) (Aug 13th, 1984)
    * `ROI_2022_nirband5.tif` - clip of Landsat 9 OLI Scene 230/69, Near Infrared Band (B5)(Aug 14th, 2022)
    * `ROI_2022_redband4.tif` - clip of Landsat 9 OLI Scene 230/69, Red Band (B4) (Aug 14th, 2022)
* Org: The US Geological Survey (USGS)
* Resource: LandsatLook, a portal that allows rapid online viewing and access to the USGS Landsat Collection 2 data -> https://landsatlook.usgs.gov/


**Extra info on Spectral Remote Sensing**
* National Ecological Observatory Network (NEON), "Mapping the Invisible - Intro to Spectral Remote Sensing" (short video) -> https://youtu.be/3iaFzafWJQE
* NASA Remote Sensing primer -> https://www.earthdata.nasa.gov/learn/backgrounders/remote-sensing 
* NASA Applied Remote Sensing Training Program (ARSET), Remote Sensing Fundamentals -> https://appliedsciences.nasa.gov/sites/default/files/2022-11/Fundamentals_of_RS_Edited_SC.pdf

## A. Set-up Jupyter Notebook & satellite data

>**A0.** Import required packages and submodule.
>```
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
```

>**A1.** (OPTIONAL) For autocompletion, or if it's not working, try running this magic command.
```
%config Completer.use_jedi = False
```

>**A2.** Read-in the 4 satellite image files and assign each to a variable of the same name.
>
>```
ROI_1984_nirband4 = np.asarray(Image.open("ROI_1984_nirband4.tif"), dtype=np.float32)
ROI_1984_redband3 = np.asarray(Image.open("ROI_1984_redband3.tif"), dtype=np.float32)
ROI_2022_nirband5 = np.asarray(Image.open("ROI_2022_nirband5.tif"), dtype=np.float32)
ROI_2022_redband4 = np.asarray(Image.open("ROI_2022_redband4.tif"), dtype=np.float32)
```

---
## B. Inspect the satellite data

<font color="green">***B. Intro***   
*- Raster data is one of two main types of geospatial data. The other is vector data, such as the points and polygons handled using geopandas.*    
*- Data stored in raster format are typically in a grid/matrix structure, and render as pixels (picture elements).*      
*- For geospatial raster data, each pixel relates to some area on Earth, and each pixel value is a measurement of some real-world phenomenon.*       
*- The 4 files are geospatial raster datasets, and the pixel values they contain are satellite measurements of amounts of light reflected from a single band of the electromagnetic spectrum/defined range of wavelengths, i.e. spectral reflectance values.*    
*- Specifically, each single-band raster contains measurements of either `Red` band or `Near Infrared` (NIR) reflected light.*     

<font color="green">***CAVEAT!! HIGH-LEVEL EXPLORATION ONLY!!***    
*- There are differences in both `Red` and `NIR` band coverage between the two Landsat sources of the files, Landsat 5 TM and Landsat 9 OLI, which are older and new sensors respectively - see comparison table below for details extracted from the USGS band designation doc -> https://www.usgs.gov/faqs/what-are-band-designations-landsat-satellites).*    
    
    
|                     | Red Band Wavelength | NIR Band Wavelength |
| ------------------- | -------------|---------------|
|  Landsat 5 TM|  0.63 - 0.69 |  0.76 - 0.90  |
|  Landsat 9 OLI|  0.64 - 0.67 |  0.85 - 0.88  |
    
    
<font color="green">*- As the two sensors have not, therefore, collected like-for-like data, expect some limitation on the comparability of data outputs derived from these different sources.*

>**B0.** Check the dimensions of any of the 4 rasters, i.e. `ROI_1984_redband3`, `ROI_1984_nirband4`, `ROI_2022_redband4`, or `ROI_2022_nirband5`.
>
>**Tech Note:** The original GeoTIFF files have 3 dimensions, i.e. (<no. of bands>, <no. of rows>, <no. of columns>). However given single-band rasters the band dimension has length 1, thus we handle as a 2 dimensional array.
>```
ROI_1984_redband3.shape
```

>**B1.** Check the `dtype` of any of the rasters.
>
>**Tech Note:** The original data downloaded from LandsatLook has the `dtype` `uint16`. The read-in rasters are now `floats` as a result of specifying `NoData` values to be masked, i.e. converted to `NaN`. Read more on the raw satellite data, e.g. from here -> https://yceo.yale.edu/how-convert-landsat-dns-top-atmosphere-toa-reflectance
>```
ROI_1984_redband3.dtype
```

>**B2.** Try rendering the Red band reflectance values of the ROI in 1984 using the matplotlib `matshow()` function. Pass `"Reds_r"` as the optional `cmap` argument, and adjust the size and aspect of the plot (to be smaller than the default so as to be faster).
>
>**Tech Note:** The areas where more light was reflected (with higher spectral reflectance values) should be coloured as brighter pixels and vice versa, hence the reversed cmap `"Reds_r"`.
>```
plt.matshow(ROI_1984_redband3, cmap="Reds_r")
```

<font color="green">***B2. Comment***    
*- There is a visible orthogonal pattern that looks like the "Fishbone" Amazon deforestation pattern -> https://landsat.visibleearth.nasa.gov/view.php?id=145888*

>**B4.** Have a look at some actual reflectance values. Extract a sample at the top-left corner of the `ROI_1984_redband3` raster, a subarray specifically the first 5 rows and 5 columns.    
>
>```
ROI_1984_redband3[:5, :5]    # Reminder that 2D array indexing is in the format [<rows>, <columns>]
```

<font color="green">***B4. Comments***     
*- Each value is the amount of Red band reflectance from a specific 30m x 30m area at the Brazil-Bolivia border that Landsat 5 TM measured on August 13th 1984.*    
*- 30m represents the spatial resolution of our rasters, a metric indicating the ground surface area that forms a single pixel. (Satellite) sensors can have higher or lower spatial resolution, e.g. MODIS where 250m-1km squared is covered by each pixel.*

>**B5.** Try rendering this subarray of Red band reflectance values using matplotlib's `matshow()` function.
>```
plt.matshow(ROI_1984_redband3[:5, :5], cmap="Reds_r")
```

>**B6.** (OPTIONAL) Extract and plot a subarray of `ROI_1984_redband3` values as in **B5.** but this time annotate each pixel with it's (reflectance) value.
>```
array = ROI_1984_redband3[:5, :5]
fig, ax = plt.subplots(figsize=(5,5))    # Adjust the size to fit the subarray size and/or adjust the annotations in the `plt.text()` call
ax.matshow(array, cmap="Reds_r")
for y in range(array.shape[0]):
    for x in range(array.shape[1]):
        plt.text(x,y,array[y,x], bbox=dict(facecolor='beige'), horizontalalignment="center")
plt.show()
```

>**B7.** As the area of the ROI in 1984 covered by the pixels in the top-left corner is not overly interesting (!), let's have a look at the top-right corner instead.    
> Start by visualising a 5x5 matrix, then keep increasing the size of the row/column slices, e.g. up to 500x500.
>```
plt.matshow(ROI_1984_redband3[:5, -5:], cmap="Reds_r")    # Use negative indices for ease
```

<font color="green">***B7. Comment***     
*- NASA has a (legacy) collection of satellite image showcasing interesting Agricultural Patterns -> https://earthobservatory.nasa.gov/images/6605/agricultural-patterns*

---
## C. Prepare to calculate a spectral vegetation index

<font color="green">***C. Intro***    
*- A Spectral Index or Ratio is an indicator of some phenomena, such as biophysical or biochemical properties, calculated using the different reflectance properties of different objects.*    
*- Landsat data products like Burn Ratios and Moisture Index are examples of spectral indices.*    
*- One type of spectral indices relevant to Deforestation are Vegetation Indices (VI), the most common of which is the Normalized Difference Vegetation Index (NDVI), used to quantify vegetation greenness.*    

<font color="green">*- NDVI utilises certain reflectance properties of plants: chlorophyll strongly absorbs Red light, whilst leaves/plant structure strongly reflects NIR light.*      
*- The formula to calculate NDVI therefore requires the reflectance values in the Red and NIR bands:*

$$
  NDVI = \frac{(Near Infrared - Red)}{(Near Infrared + Red)}
$$      

<font color="green">*- By design, NDVI values therefore range from -1.0 to +1.0. Negative values to 0 indicate no green leaves, whilst values approaching +1.0 indicate highest density of green leaves. (See NASA NDVI training, slide 10-11 -> https://appliedsciences.nasa.gov/sites/default/files/2020-11/ndvipart1.pdf)*        
*- In Section D. we will calculate NDVI values for our ROI to help detect how the existing tropical dry forest cover in 1984 had changed by 2022, as likely Deforestation.*     
*- Note that NDVI is often a pre-calculated data product, and is available from the LandsatLook portal - however for learning purposes we will construct our own.*

>**C0.** In advance of calculating NDVI values using the above raster math formula, render the Red and NIR rasters of the ROI from 2022 as subplots.
>
>**Code Detail:** Use functionality provided by xarray via the `robust` parameter to compute plot colour limits excluding outliers, i.e. using the 2nd and 98th percentiles of the data -> https://docs.xarray.dev/en/stable/user-guide/plotting.html#robust
>```
fig, ax = plt.subplots(1,2, figsize=(14,5))
ax[0].matshow(ROI_1984_redband3, cmap="Reds_r")
ax[1].matshow(ROI_1984_nirband4, cmap="Blues_r")
plt.show()
```

---
## D. Calculate Normalized Difference Vegetation Index (NDVI)

>**D0.** Calculate NDVI values for the ROI in 1984 by performing the required raster math with `ROI_1984_nirband4` and `ROI_1984_redband3`, and assign to `NDVI_1984`.
>```
NDVI_1984 = (ROI_1984_nirband4 - ROI_1984_redband3) / (ROI_1984_nirband4 + ROI_1984_redband3)
```

>**D1.** Sense-check some of the `NDVI_1984` values, must be between -1.0 and +1.0.
>```
NDVI_1984[:2, :2]
```

>**D2.** Calculate NDVI values for the ROI in 2022 by performing the required raster math with `ROI_2022_nirband4` and `ROI_2022_redband3`, and assign to `NDVI_2022`.
>```
NDVI_2022 = (ROI_2022_nirband5 - ROI_2022_redband4) / (ROI_2022_nirband5 + ROI_2022_redband4)
```

>**D3.** Also sense-check some of the `NDVI_2022` values.
>```
NDVI_2022[:2, :2]
```

>**D4.** (OPTIONAL) Export NDVI arrays as binary files in numpy `.npy` format. Use `np.load()` to read.
>```
np.save("NDVI_1984", NDVI_1984)
np.save("NDVI_2022", NDVI_2022)
```

---
## E. Create & Interpret NDVI maps

>**E0.** The NDVI datasets are still geospatial rasters, i.e. grids of pixels, and still each linked to 30 square meters on Earth. However, the value of these pixels indicate the density of green leaves covering that place on Earth, on a scale of -1.0 to +1.0, (i.e. not measurements of spectral reflectance). Let's therefore render both NDVI rasters to help discover how dense vegetation was in our ROI in 1984 compared to 2022.

>**Code Detail:** Use manually pre-calculated `vmin` and `vmax` args to scale the (false) color to the actual range of our NDVI values.
>```
fig, ax  = plt.subplots(1,2, figsize=(14,5))
plot0 = ax[0].matshow(NDVI_1984, cmap="terrain_r", vmin=-0.15, vmax=0.75)
fig.colorbar(plot0, ax=ax[0])
plot1 =ax[1].matshow(NDVI_2022, cmap="terrain_r", vmin=-0.15, vmax=0.75)
fig.colorbar(plot1, ax=ax[1])
plt.show()
```

<font color="green">***E0. Comments***    
*- **CAVEAT REMINDER!!** The difference in Landsat 5 TM and Landsat 9 OLI's band coverage affects the comparability of the respective NDVI computed, as derivative data products.*    
*- However, these NDVI maps do provide guidance about the potential LCLU in this Amazon River basin region in 1984 and 2022:*    
 
*"Areas of barren rock, sand, or snow usually show very low NDVI values (for example, 0.1 or less). Sparse vegetation such as shrubs and grasslands or senescing crops may result in moderate NDVI values (approximately 0.2 to 0.5). High NDVI values (approximately 0.6 to 0.9) correspond to dense vegetation such as that found in temperate and tropical forests or crops at their peak growth stage."* (https://www.usgs.gov/special-topics/remote-sensing-phenology/science/ndvi-foundation-remote-sensing-phenology)

<font color="green">*- Other Remote Sensing considerations: Controlling for satellite band designations, spectral reflectance and therefore also derived products like spectral indices, are sensitive to different angles of observation and differences in the atmostphere during the two satellite overpasses, amongst many other factors.*

>**E1.** So far in this Exploration we've been handling continuous raster data. However, in preparation for developing a thematic Land Cover map next section, let's start to consider how we could meaningfully group, or bin, pixels into NDVI ranges associated with specific land cover. Check the distribution of both sets of NDVI values through quick histogram plotting. 
>```
plt.figure()
plt.hist(NDVI_1984.flatten(), color="royalblue")
plt.hist(NDVI_2022.flatten(), color="m", alpha=0.5);   # The semi-colon tidies the plot by suppresses some extraneous info
```

---
## F. Classify Land Cover using NDVI & make thematic map

<font color="green">***F. Intro***    
*- LEARNING NOTE: This section is designed to **introduce** the idea of pixel-based Image Classification of Land Cover using Satellite Imagery in order to create thematic maps, and **does not represent best-practice**! Check out NASA training material on this topic, e.g. -> https://appliedsciences.nasa.gov/sites/default/files/Session%20One%20-%20Introduction%20to%20Land%20Cover%20Classification%20and%20QGIS.pdf*      
*- We will be executing an experimental process, trying to develop a meaningful thematic Land Cover map of our ROI in 2022, leveraging known NDVI values of certain land cover, and the distribution of empirical NDVI values.*      
*- Central to this process is performing pixel-based classification. We will have a go at producing a map that describes 4 land cover types, by implementing the below experimental classification scheme whereby every pixel will be assigned a class, based on it's NDVI value:*

> 
|       Class                   | NDVI   | 
| ----------------------------- | ------ |
| Rock/Bare Soil/Water/Concrete | -1.0 <= NDVI < 0.15 |
| Sparse/Senescing Veg1         | 0.15 <= NDVI < 0.25 |
| Sparse/Senescing Veg2         | 0.25 <= NDVI < 0.4  |
| Denser Veg                    | 0.4  <= NDVI <= 1.0 |
>


>**F0.** Check the type of the `NDVI_2022` object.
>```
type(NDVI_2022)
```

>**F1.** Make a copy of `NDVI_2022` named `LCLU`.
>```
LCLU = NDVI_2022.copy()
```

>**F2.** Our approach to implementing the pixel classification involves manually binning the subset of pixels that belong in each class (so the process is more transparent).  Try calling numpy's `where()` function in order to test replacing the NDVI values in `LCLU` which are `< 0.3` with `99`.
>```
np.where(LCLU < 0.3, 99, LCLU)
```

>**F3.** Build the final array, `LCLU_4`, through creating a series of incrementally updated arrays, that replace the pixel values assigned to each class with `100`, `200`, `300`, and `400` respectively.
>
>**Tech Note:**    
>1.0 <= NDVI <0.15 becomes 100 - Rock/Bare Soil/Water/Concrete class    
>0.15 <= NDVI <0.25 becomes 200 - Sparse/Senescing Veg1 class    
>0.25 <= NDVI <0.4 becomes 300 - Sparse/Senescing Veg2 class     
>0.4 <= NDVI <=1.0 becomes 400 - Denser Veg  class    
>```
LCLU_1 = np.where(LCLU < 0.15, 100, LCLU)
LCLU_2 = np.where(LCLU_1 < 0.25, 200, LCLU_1)
LCLU_3 = np.where(LCLU_2 < 0.4, 300, LCLU_2)
LCLU_4 = np.where(LCLU_3 <= 1, 400, LCLU_3)
```

>**F4.** Review the fully classified raster `LCLU_4`, representing our Land Cover map, by generating a quick plot.
>```
LCLU_4_plot = plt.matshow(LCLU_4)
fig.colorbar(LCLU_4_plot)
plt.show()
```

>**F5.** Let's prepare to construct an upgraded version of this Land Cover map. Import the `ListedColormap` class from matplotlib.colors.
>```
from matplotlib.colors import ListedColormap
```

>**F6.** Create a custom cmap consisting of 4 appropriate named colours called `LCLU_cmap`. The colours will be used in turn, starting from the lowest data values then ascending, i.e. 100 to 400.
```
LCLU_cmap = ListedColormap( ["sandybrown", "gold", "yellowgreen", "forestgreen"] )
```

>**F7.** Make the upgraded Land Cover map of the ROI from 2022, using `LCLU_cmap` and other matplotlib options to fine-tune the plot.
>```
fig, ax = plt.subplots(figsize=(8,6))
classed_plot = ax.matshow(LCLU_4, cmap=LCLU_cmap)
cbar = fig.colorbar(classed_plot)
cbar.set_ticks([100,200,300,400])
cbar.set_ticklabels(["Rock/Bare Soil/Water/Concrete", "Sparse/Senescing Veg1", "Sparse/Senescing Veg2", "Denser Veg"])
#plt.tight_layout()
#plt.savefig("<SomeFilename>.png", dpi=600)
plt.show()
```